# Optimisation Run Flexibility
It is possible to change the configuration of the algorithm part way through the optimization process, or even to switch algorithms completely.  
This allows an optimization process to be configured to be more exploritative early on, to explore the whole design space, then later to be more exploitative, to home in in exact optimal solutions.
Doing so requires using [Platypus](https://platypus.readthedocs.io/en/latest/) algorithms directly, instead of the algorithm wrappers provided through the BESOS optimizer module.

In [1]:
import platypus

from besos import eppy_funcs as ef
from besos.parameters import expand_plist
from besos.evaluator import EvaluatorEP
from besos.problem import EPProblem
from besos import optimizer

First we create an example problem, see [here]BuildingOptimization.ipynb) for details.

In [2]:
idf = ef.get_idf()
parameters=expand_plist(
    {'NonRes Fixed Assembly Window':
     {'UFactor':(0.1,5),
      'Solar Heat Gain Coefficient':(0.01,0.99)
     },
     'Mass NonRes Wall Insulation':{'Thickness':(0.01,0.09)},
    })
objectives = ['Electricity:Facility', 'Gas:Facility']
problem = EPProblem(parameters, objectives)
evaluator = EvaluatorEP(problem, idf)

Next we set up NSGA-II as the first algorithm, a good general purpose multi-objective genetic algorithm.  
The `to_platypus` shortcut converts the Evaluator object to a `platypus.Problem` object.

In [3]:
platypus_problem = evaluator.to_platypus()
algorithm = platypus.NSGAII(problem=platypus_problem)

Now we can run the algorithm for a lot of generations, and pause it at some point.  
Use the **stop button** at the top of the notebook to interrupt the following cell.  
Note: The output from the next cells will vary from run to run, due to the randomness of the underlying algorithm as well as the amount of time this cell is run for.

In [4]:
try:
    algorithm.run(10)
except KeyboardInterrupt:
    print('Algorithm interrupted')
algorithm.population[:5]

[Solution[2.213855058719472,0.6363995124730418,0.05052880146849404|1936010631.218226,2434618616.136553|0],
 Solution[4.925557981680829,0.4412852809214969,0.06977088347725961|1893368110.6571507,2696496418.7657657|0],
 Solution[4.7060110431146835,0.6806276770141859,0.023322597146269802|1995725412.1155868,3044778824.3118696|0],
 Solution[3.765857877806432,0.1552088397670927,0.022016834149865015|1810620642.7659373,3012916012.399791|0],
 Solution[1.331891701055805,0.2727493231996027,0.012779804276336302|1860819888.7077937,2852800484.7137547|0]]

Now we want to continue from where the first algorithm left off, running `EpsMOEA` for 10 evaluations.  
In order to make the population carry over, we use the `InjectedPopulation` generator, then run the second algorithm.

If we had let the first algorithm finish, we could use `algorithm.result` instead of `algorithm.population` to use the solutions found by the first algorithm as a starting point for the next.

In [5]:
generator = platypus.InjectedPopulation(algorithm.population)
alg2 = platypus.EpsMOEA(problem=platypus_problem, generator=generator, epsilons=3, population_size=10)
alg2.run(10)

Now we convert the solutions to a dataframe using the BESOS helper function and display them.

In [6]:
optimizer.solutions_to_df(alg2.result, problem, parts=['inputs', 'outputs'])

,UFactor,Solar Heat Gain Coefficient,Thickness,Electricity:Facility,Gas:Facility,pareto-optimal
0,1.308471,0.074466,0.047176,1.778193e+09,2.359371e+09,True
1,2.841974,0.144283,0.084541,1.777331e+09,2.416748e+09,True
2,0.413365,0.515070,0.082873,1.892471e+09,1.956768e+09,True
3,0.614166,0.170777,0.083544,1.796432e+09,2.027675e+09,True
4,2.899266,0.065677,0.088525,1.751869e+09,2.418912e+09,True
5,0.190484,0.799682,0.072841,2.015778e+09,1.907172e+09,True
6,2.606209,0.014942,0.065966,1.748498e+09,2.465901e+09,True
